In [ ]:
from os.path import join
import numpy as np
import pandas as pd
from numpy import linalg as LA
import torchvision
import sys
import torch
import os
import scipy.io
from pathlib import Path
import warnings
warnings.filterwarnings('ignore')

class Namespace:
    def __init__(self, **kwargs):
        self.__dict__.update(kwargs)

## GPU or CPU
GPU = True
if GPU:
    torch.backends.cudnn.enabled = True
    torch.backends.cudnn.benchmark = True
    os.environ['CUDA_VISIBLE_DEVICES'] = '0'
    print(f"GPU(s): {torch.cuda.device_count()}x {torch.cuda.get_device_name()}")
    dtype = torch.cuda.FloatTensor
else:
    dtype = torch.FloatTensor
    print("CPU")
eps = torch.finfo(torch.float32).eps
torch.cuda.empty_cache()

In [ ]:
# Read Dataset

def readDataset(selected):
    # arr = os.listdir('Dataset')

    edgelist = np.genfromtxt(f'Dataset/Celegansneural.edges').astype(int)
    e = len(edgelist)
    n = np.max(edgelist)
    A = torch.zeros([n, n])
    Atr = torch.zeros([n, n])

    # Percent of delete edges
    per = 0.9
    oe = np.round(per * e).astype(int)

    train = np.random.choice(e, oe, replace=False)

    for i in range(e):
        A[edgelist[i, 0] - 1, edgelist[i, 1] - 1] = 1
        A[edgelist[i, 1] - 1, edgelist[i, 0] - 1] = 1

    for i in train:
        Atr[edgelist[i, 0] - 1, edgelist[i, 1] - 1] = 1
        Atr[edgelist[i, 1] - 1, edgelist[i, 0] - 1] = 1

    Ate = A - Atr
    dataset_name = 'Celegansneural'

    print("Dataset_name: ",dataset_name,"\n#node => ",n, "#edge => ",e,'\n')
    return A, Atr, Ate, dataset_name, n, e

In [1]:
# precision Mesure
def precision(V, Ate,Apred):
    
    n = Ate.shape[0]
    Ap = (1-V) * Apred
    L1 = torch.sum(Ate).int()
    
    ap = torch.reshape(Ap, (n**2,1))
    ate = torch.reshape(Ate, (n**2,1))
    values, indices = ap.topk(k=L1,dim=0) 
    Ind = indices[values!=0]
    app = torch.zeros(n**2,1)
    app[Ind] = 1
    
    l1 = torch.sum(app.cuda() * ate.cuda())
    prec = l1/L1
    return prec.cpu()


In [ ]:
# baseline methods Similarity

def jaccard_similarity(V):
    n = V.shape[0]
    u = V @ V.T
    a = torch.sum(V,1)
    b = a.repeat(n,1)
    J = (u)/(b+b.T-u)
    J = J * (1 - torch.eye(n).cuda())
    return torch.nan_to_num(J)

def Common_neighbours(V):
    return V @ V.T

def Resource_Allocation(V,degree):
    degreeR = degree.repeat(n,1)
    dd = degreeR * V
    mask = (dd != 0)
    DD = torch.zeros([N,N]).cuda()
    O = torch.ones([N,N]).cuda()
    DD[mask] = O[mask] / dd[mask]
    RA = DD @ V.T
    return RA

def AdamicAdar(V,degree):
    degreeR = degree.repeat(n,1)
    dd = torch.log2(degreeR * V)
    mask = (dd != 0)
    DD = torch.zeros([N,N]).cuda()
    O = torch.ones([N,N]).cuda()
    DD[mask] = O[mask] / dd[mask]
    AA = DD @ V.T
    return AA

# Preferential Attachment (PA)
def Preferential_Attachment(V):
    V = V.to(torch.float64)
    a = torch.sum(V,1)
    a1 = a.reshape([len(a), 1])
    AA = a1 @ a1.T
    return AA

In [ ]:
# Similarity Based Methods : jaccard , Common_neighbours , Resource_Allocation(RA) , AdamicAdar

A, Atr, Ate, dataset_name, n, e = readDataset(1)
[F,N] = A.shape

A = A.type(dtype)
Ate = Ate.type(dtype)
V = Atr.type(dtype)
degree = torch.sum(V , 0)

Apred = jaccard_similarity(V)
prec = precision(V, Ate, Apred)
print('Jaccard Precision= ',prec)

Apred = Common_neighbours(V)
prec = precision(V, Ate, Apred)
print('CN Precision= ',prec)

Apred = Resource_Allocation(V,degree)
prec = precision(V, Ate, Apred)
print('RA Precision= ',prec)

Apred = AdamicAdar(V,degree)
prec = precision(V, Ate, Apred)
print('AA Precision= ',prec)

Apred = Preferential_Attachment(V)
prec = precision(V, Ate, Apred)
print('PA Precision= ',prec)